In [5]:
import json
Data = []
with open(r"D:\1Acyn\FUDAN\2022学年\知识计算\project\NER_ZH\data\cluener_public\train.json", 'r') as f:
    for line in f:
        try:
            Data.append(json.loads(line.rstrip(';\n')))
        except ValueError:
            print ("Skipping invalid line {0}".format(repr(line)))


{'name': {'叶老桂': [[9, 11]]}, 'company': {'浙商银行': [[0, 3]]}}

In [8]:
from transformers import AdamW, get_linear_schedule_with_warmup, BertTokenizer, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

c:\Users\ASUS\anaconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
tokenizer(''.join(Data[1]['text']))

{'input_ids': [101, 4495, 4495, 679, 2622, 9541, 8798, 4495, 1265, 4312, 4060, 6375, 872, 1856, 2486, 4312, 2812, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
Data[0]['label']

{'name': {'叶老桂': [[9, 11]]}, 'company': {'浙商银行': [[0, 3]]}}

In [19]:
Data[1]['label']

{'game': {'CSOL': [[4, 7]]}}

In [30]:
import numpy as np

label_list = ['address', 'book', 'company', 'game', 'government', 'movie', 'name', 'organization', 'position', 'scene']
label_map = {label: i for i, label in enumerate(label_list)}

max_seq_length = 128
word_areas = np.zeros((max_seq_length, 3))
total_tags = np.zeros((max_seq_length))
total_areas = np.zeros((max_seq_length, max_seq_length))

pos = 0
for k in Data[0]['label'].keys():
    for v in Data[0]['label'][k].values():
        start = v[0][0]
        end = v[0][1]
        total_areas[pos][start : (end + 1)] += 1 / (end - start + 1)
        total_tags[pos] = label_map[k] + 1
        pos += 1
        for i in range(start, end + 1):
            word_areas[i, 0] = 1
            word_areas[i, 1] = start - i
            word_areas[i, 2] = end -i

print(total_tags)
print(label_map)

[7. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
{'address': 0, 'book': 1, 'company': 2, 'game': 3, 'government': 4, 'movie': 5, 'name': 6, 'organization': 7, 'position': 8, 'scene': 9}


In [22]:
for v in Data[0]['label']['name'].values():
    print(v[0])

[9, 11]
